In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, roc_curve, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay

original = pd.read_csv("dados.csv")

In [ ]:
le = preprocessing.LabelEncoder()

modificado = pd.DataFrame()
for x in original:
    if original[x].dtypes==object:
        modificado[x] = le.fit_transform(np.array(original[x]))
    else:
        modificado[x] = np.array(original[x])

clm = list(modificado.columns)
X = np.array(modificado[clm[0:-1]])
y = np.array(modificado[clm[-1]])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

escala = preprocessing.MinMaxScaler(feature_range=(0, 1))
clfNN = MLPClassifier(hidden_layer_sizes=(3, 10,10,10), alpha=1e-5,verbose=True, learning_rate="adaptive",max_iter=500)
clasNN = make_pipeline(escala, clfNN)
clasNN.fit(X_train,y_train)
clasNN.score(X_test,y_test)
yp = clasNN.predict(X_test)

In [5]:
from sklearn.utils import resample

le = preprocessing.LabelEncoder()

modificado = pd.DataFrame()
for x in original:
    if original[x].dtypes==object:
        modificado[x] = le.fit_transform(np.array(original[x]))
    else:
        modificado[x] = np.array(original[x])

y = modificado.COMPARECEU_AGENDAMENTO_EXAME
X = modificado.drop('COMPARECEU_AGENDAMENTO_EXAME', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X = pd.concat([X_train, y_train], axis=1)
NAO = X[X.COMPARECEU_AGENDAMENTO_EXAME==0]
SIM = X[X.COMPARECEU_AGENDAMENTO_EXAME==1]

NAO_upsampled = resample(NAO,
                          replace=True, # sample with replacement
                          n_samples=len(SIM), # match number in majority class
                          random_state=42)

upsampled = pd.concat([SIM, NAO_upsampled])
upsampled.COMPARECEU_AGENDAMENTO_EXAME.value_counts()

y_train_md = np.array(upsampled.COMPARECEU_AGENDAMENTO_EXAME)
X_train_md = np.array(upsampled.drop('COMPARECEU_AGENDAMENTO_EXAME', axis=1))


escala = preprocessing.MinMaxScaler(feature_range=(0, 1))
clfNN = MLPClassifier(hidden_layer_sizes=(3, 10,10,10), alpha=1e-5,verbose=True, learning_rate="adaptive",max_iter=500)
clasNN = make_pipeline(escala, clfNN)
clasNN.fit(X_train_md,y_train_md)
sc = clasNN.score(np.array(X_test),np.array(y_test))
yp = clasNN.predict(np.array(X_test))
f1sc = f1_score(yp,np.array(y_test))
print(f1sc,sc)

Iteration 1, loss = 0.72851124
Iteration 2, loss = 0.70275743
Iteration 3, loss = 0.69059254
Iteration 4, loss = 0.68745636
Iteration 5, loss = 0.68388690
Iteration 6, loss = 0.67870323
Iteration 7, loss = 0.67172275
Iteration 8, loss = 0.66624122
Iteration 9, loss = 0.66183168
Iteration 10, loss = 0.65962970
Iteration 11, loss = 0.65830348
Iteration 12, loss = 0.65730169
Iteration 13, loss = 0.65623201
Iteration 14, loss = 0.65495652
Iteration 15, loss = 0.65368901
Iteration 16, loss = 0.65244944
Iteration 17, loss = 0.65130142
Iteration 18, loss = 0.65029092
Iteration 19, loss = 0.64901062
Iteration 20, loss = 0.64899104
Iteration 21, loss = 0.64690093
Iteration 22, loss = 0.64587004
Iteration 23, loss = 0.64471503
Iteration 24, loss = 0.64357314
Iteration 25, loss = 0.64255999
Iteration 26, loss = 0.64187074
Iteration 27, loss = 0.64069878
Iteration 28, loss = 0.64005763
Iteration 29, loss = 0.63909391
Iteration 30, loss = 0.63860926
Iteration 31, loss = 0.63785651
Iteration 32, los